In [1]:
import pandas as pd
import numpy as np
import time 
def max_exp(l,tpm,tpmi,tissue):
    ex=0
    for i in l:
        if i in tpmi:
            ex=1
    if ex==1:
        em=0
        for i in l:
            if i in tpmi:
                e=tpm.loc[i,tissue]
                if e>em:
                    em=e
    else:
        em=np.nan
    return em
t=time.time()
source='data/source/'
#mane.gz from MANE Sellect for TSS
h={
    #'ez':'https://www.encodeproject.org/files/ENCFF414CAB/@@download/ENCFF414CAB.bed.gz' #H1 EZH Bed
    'h1ez':'https://www.encodeproject.org/files/ENCFF109KCQ/@@download/ENCFF109KCQ.bigWig', #H1 EZH bw
    'h1suz':'https://www.encodeproject.org/files/ENCFF609MMJ/@@download/ENCFF609MMJ.bigWig', #h1 suz
    'h1':'http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H1/tracks_hg38/Human_H1.meth.bw',
    'h9':'http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H9/tracks_hg38/Human_H9.meth.bw',
    'h1_hmr':'http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H1/tracks_hg38/Human_H1.hmr.bb',
}
t4={
    't4y':'http://smithdata.usc.edu/methbase/data/Heyn-Human-NewbornCentenarian-2012/Human_CD4T-Newborn/tracks_hg38/Human_CD4T-Newborn.meth.bw',
    't4o':'http://smithdata.usc.edu/methbase/data/Heyn-Human-NewbornCentenarian-2012/Human_CD4T-100yr/tracks_hg38/Human_CD4T-100yr.meth.bw',
    't4y_hmr':'http://smithdata.usc.edu/methbase/data/Heyn-Human-NewbornCentenarian-2012/Human_CD4T-Newborn/tracks_hg38/Human_CD4T-Newborn.hmr.bb',    
    't4o_hmr':'http://smithdata.usc.edu/methbase/data/Heyn-Human-NewbornCentenarian-2012/Human_CD4T-100yr/tracks_hg38/Human_CD4T-100yr.hmr.bb' ,   
}

tmp='https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'

geo={
'pbmc_neo':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE140nnn/GSE140730/matrix/GSE140730-GPL20795_series_matrix.txt.gz', #neonatal wg blood hg38
}


man450_19='https://webdata.illumina.com/downloads/productfiles/humanmethylation450/humanmethylation450_15017482_v1-2.csv'
man450_38='https://zhouserver.research.chop.edu/InfiniumAnnotation/20210615/HM450/HM450.hg38.manifest.gencode.v36.tsv.gz'
man_epic='https://webdata.illumina.com/downloads/productfiles/methylationEPIC/infinium-methylationepic-v-1-0-b5-manifest-file-csv.zip'
array_gses={
    'GSE76626':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76626/suppl/GSE76626_ChIP-Seq_RPM.txt.gz', ## EZH hg38
    'GSE103657':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE103nnn/GSE103657/suppl/GSE103657_beta_values.txt.gz', #neo
    'GSE103657m':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE103nnn/GSE103657/matrix/GSE103657_series_matrix.txt.gz', 
    'GSE152027':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE152nnn/GSE152027/suppl/GSE152027_IOP_processed_signals.csv.gz', # Schizophrenia ML
    'GSE152027m':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE152nnn/GSE152027/matrix/GSE152027_series_matrix.txt.gz',    
    'GSE40279':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE40nnn/GSE40279/matrix/GSE40279_series_matrix.txt.gz', # Hannum
    'GSE42861':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE42nnn/GSE42861/matrix/GSE42861_series_matrix.txt.gz', # arthritis
    'GSE55763':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE55nnn/GSE55763/suppl/GSE55763_normalized_betas.txt.gz', # quality 2,700
    'GSE55763m':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE55nnn/GSE55763/matrix/GSE55763_series_matrix.txt.gz',    
    'GSE132203':'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE132nnn/GSE132203/suppl/GSE132203_Geo_Submission_GTPEpic.csv.gz',
}

In [15]:
#!wget -nc -q {h['ez']} -P {source}
!wget -nc -q {h['h1ez']} -P {source}
!wget -nc -q {h['h1suz']} -P {source}
!wget -nc -q {h['h1']} -P 'data/meth/h/'
!wget -nc -q {h['h9']} -P 'data/meth/h/'
!wget -nc -q {h['h1_hmr']} -P 'data/meth/h/'
!wget -nc -q {h['t4y_hmr']} -P 'data/meth/t4/'
!wget -nc -q {h['t4o_hmr']} -P 'data/meth/t4/'
!wget -nc -q {t4['t4y']} -P 'data/meth/t4/'
!wget -nc -q {t4['t4o']} -P 'data/meth/t4/'

In [4]:
!wget -nc -q {tmp} -P {source}

In [5]:
!wget -nc -q {geo['pbmc_neo']} -P {source}pbmc_neo

In [6]:
neo_=source+'pbmc_neo/GSE140730-GPL20795_series_matrix.txt.gz'
neo=pd.read_csv(neo_,sep='\t',skiprows=31).T
neo=neo[neo[9]=='diagnosis: TD'] #typical development
GSE140730=neo[43].to_dict()
for gse,link in GSE140730.items():
    !wget -nc -q "{link}" -O "{source}pbmc_neo/{gse}.txt.gz"

In [7]:
!wget -nc -q {man450_19} -P {source}array
!wget -nc -q {man450_38} -P {source}array
!wget -nc -q {man_epic} -P {source}array

In [8]:
man450_of_='data/source/array/humanmethylation450_15017482_v1-2.csv'
meth450_hg38_='data/source/array/HM450.hg38.manifest.gencode.v36.tsv.gz'
cols=['IlmnID','CHR','UCSC_RefGene_Name']
man=pd.read_csv(man450_of_,skiprows=7,index_col=0,low_memory=False,engine='c',usecols=cols)
man=man[(~man.CHR.isna())&(~man.CHR.isin(['X','Y','M']))]
man38=pd.read_table(meth450_hg38_,index_col='probeID')
man38=man38[(~man38.CpG_chrm.isna())&(~man38.CpG_chrm.isin(['chrX','chrY','chrM']))].copy()
man38['CpG_chrm']=man38.CpG_chrm.str[3:].astype(float)
man38['g']=man38.CpG_chrm.astype(int)*10**9+man38.CpG_beg.astype(int)
man=man.merge(man38['g'],left_index=True,right_index=True)
gse='GSE103657' #neonatal
df=pd.read_csv('data/array/'+gse+'.csv',index_col=0,engine='c')
print(time.time()-t)
df=df.T
print(df.shape)
man=man.merge(df.median().rename('n'),left_index=True,right_index=True)

1518.2068514823914
(438, 407850)


(407839, 4)

In [15]:
tmp_='https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'
tpm=pd.read_csv(tmp_,sep='\t',skiprows=2,index_col=0)
tpm=tpm.groupby('Description').mean()
tpmi=tpm.index
man['gs']=man.UCSC_RefGene_Name.apply(lambda x: np.nan if pd.isnull(x) else set(x.split(';')))
man['e']=man['gs'].apply(lambda x: np.nan if pd.isnull(x) else  max_exp(list(x),tpm,tpmi,'Whole Blood'))

In [16]:
man.to_csv('data/array/man_450.csv')
man.shape

(407839, 6)

In [8]:
print(round(time.time()-t))

9
